In [1]:
import os
import datetime
import pickle
import warnings

import numpy as np
import pandas as pd

from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

from scipy.optimize import curve_fit
from scipy.stats import pearsonr, spearmanr, kendalltau

# Suppress warnings globally
warnings.filterwarnings('ignore')

In [2]:
class SVRWithVarianceThreshold(BaseEstimator, RegressorMixin):
    """Custom SVR estimator with integrated PCA preprocessing and extra features."""
    
    def __init__(self, variance_threshold=0.95, kernel='rbf', C=1, 
                 gamma='scale', epsilon=0.1, max_iter=-1, extra_features_to_use=None, random_state=None):
        self.variance_threshold = variance_threshold
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.epsilon = epsilon
        self.max_iter = max_iter
        # extra_features_to_use: list of indices [0,1] -> 0: MeanNIQE, 1: MeanSSIM
        # None or [] means no extra features
        self.extra_features_to_use = extra_features_to_use if extra_features_to_use is not None else []
        self.random_state = random_state
        
    def fit(self, X, y):
        # Separate main PCA features and extra features
        X_main = X[:, :-2]  # All main features (exclude last 2 columns)
        X_niqe = X[:, -2]   # MeanNIQE (second to last column)
        X_ssim = X[:, -1]   # MeanSSIM (last column)
        
        # Apply PCA to main features only
        self.main_scaler_ = StandardScaler()
        X_main_scaled = self.main_scaler_.fit_transform(X_main)
        
        # Determine PCA components
        pca = PCA(random_state=self.random_state)
        pca.fit(X_main_scaled)
        cumsum_variance = np.cumsum(pca.explained_variance_ratio_)
        n_components = np.argmax(cumsum_variance >= self.variance_threshold) + 1
        
        # Apply PCA with determined components
        self.pca_ = PCA(n_components=n_components, random_state=self.random_state)
        X_pca = self.pca_.fit_transform(X_main_scaled)
        
        # Apply sign correction for consistency
        self.pca_signs_ = np.sign(self.pca_.components_[:, 0])
        X_pca *= self.pca_signs_
        
        # Handle extra features based on extra_features_to_use parameter
        extra_features_list = []
        
        if 0 in self.extra_features_to_use:  # Include MeanNIQE
            self.niqe_scaler_ = StandardScaler()
            X_niqe_scaled = self.niqe_scaler_.fit_transform(X_niqe.reshape(-1, 1))
            extra_features_list.append(X_niqe_scaled)
        
        if 1 in self.extra_features_to_use:  # Include MeanSSIM  
            self.ssim_scaler_ = StandardScaler()
            X_ssim_scaled = self.ssim_scaler_.fit_transform(X_ssim.reshape(-1, 1))
            extra_features_list.append(X_ssim_scaled)
        
        # Combine PCA features with selected extra features
        if extra_features_list:
            X_extra_combined = np.hstack(extra_features_list)
            X_final = np.hstack([X_pca, X_extra_combined])
        else:
            X_final = X_pca
        
        # Train SVR model
        base_svr = SVR(kernel=self.kernel, C=self.C, 
                      gamma=self.gamma, epsilon=self.epsilon, 
                      max_iter=self.max_iter)
        self.model_ = MultiOutputRegressor(base_svr)
        self.model_.fit(X_final, y)
        return self
        
    def predict(self, X):
        # Separate main PCA features and extra features
        X_main = X[:, :-2]  # All main features (exclude last 2 columns)  
        X_niqe = X[:, -2]   # MeanNIQE (second to last column)
        X_ssim = X[:, -1]   # MeanSSIM (last column)
        
        # Apply PCA to main features
        X_main_scaled = self.main_scaler_.transform(X_main)
        X_pca = self.pca_.transform(X_main_scaled)
        X_pca *= self.pca_signs_  # Apply same sign correction
        
        # Handle extra features based on extra_features_to_use parameter  
        extra_features_list = []
        
        if 0 in self.extra_features_to_use:  # Include MeanNIQE
            X_niqe_scaled = self.niqe_scaler_.transform(X_niqe.reshape(-1, 1))
            extra_features_list.append(X_niqe_scaled)
            
        if 1 in self.extra_features_to_use:  # Include MeanSSIM
            X_ssim_scaled = self.ssim_scaler_.transform(X_ssim.reshape(-1, 1))
            extra_features_list.append(X_ssim_scaled)
        
        # Combine PCA features with selected extra features
        if extra_features_list:
            X_extra_combined = np.hstack(extra_features_list)
            X_final = np.hstack([X_pca, X_extra_combined])
        else:
            X_final = X_pca
            
        return self.model_.predict(X_final)


In [3]:
class MultiOutputSVRWithPCA:
    """
    A class for multi-output Support Vector Regression with PCA preprocessing
    and hyperparameter tuning using GridSearchCV.
    """
    def __init__(self, random_state=42):
        """Initialize the class with empty attributes."""
        self.features_df = None
        self.labels_df = None
        self.extra_features_df = None
        self.X = None
        self.y = None
        self.X_train = None  # Added train/test split attributes
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None
        self.best_params = None
        self.label_columns = ['TSV', 'B', 'SR', 'S', 'U', 'O']
        self.logistic_model = "4pl"
        self.extra_features_option = None  # Options: 'none', 'niqe', 'ssim', 'niqe+ssim'
        self.results_comparison = {}  # Store results for different combinations
        self.random_state = random_state  # Single random state for the entire pipeline
        
    def load_data(self, features_file, labels_file, extra_features_file=None):
        """Load features and labels from CSV files, including extra features."""
        print("Loading data...")
        # Load features file
        self.features_df = pd.read_csv(features_file)
        print(f"Features shape: {self.features_df.shape}")
        # Load labels file
        self.labels_df = pd.read_csv(labels_file)
        print(f"Labels shape: {self.labels_df.shape}")
        
        # Load extra features file
        if extra_features_file is not None:
            self.extra_features_df = pd.read_csv(extra_features_file)
            print(f"Extra features shape: {self.extra_features_df.shape}")
        
        # Merge dataframes on videoname
        merged_df = pd.merge(self.features_df, self.labels_df, on='videoname')
        print(f"Merged data shape: {merged_df.shape}")
        
        if extra_features_file is not None:
            # Merge with extra features
            merged_df = pd.merge(merged_df, self.extra_features_df[['videoname', 'MeanNIQE', 'MeanSSIM']], on='videoname')
            print(f"Merged with extra features shape: {merged_df.shape}")
            
            # Check for missing values in extra features
            missing_niqe = merged_df['MeanNIQE'].isna().sum()
            missing_ssim = merged_df['MeanSSIM'].isna().sum()
            if missing_niqe > 0 or missing_ssim > 0:
                print(f"WARNING: Missing values - MeanNIQE: {missing_niqe}, MeanSSIM: {missing_ssim}")
                # Fill missing values with median
                merged_df['MeanNIQE'].fillna(merged_df['MeanNIQE'].median(), inplace=True)
                merged_df['MeanSSIM'].fillna(merged_df['MeanSSIM'].median(), inplace=True)
                print("Missing values filled with median.")
        
        # Extract features (all columns except videoname and labels)
        feature_columns = [col for col in self.features_df.columns if col != 'videoname']
        X_main = merged_df[feature_columns].values
        
        # Always add extra features columns (MeanNIQE, MeanSSIM) as last 2 columns
        if extra_features_file is not None:
            X_extra = merged_df[['MeanNIQE', 'MeanSSIM']].values
            self.X = np.hstack([X_main, X_extra])
        else:
            # If no extra features, add dummy columns with zeros
            X_extra = np.zeros((X_main.shape[0], 2))
            self.X = np.hstack([X_main, X_extra])
            print("WARNING: No extra features file provided. Using zeros for MeanNIQE and MeanSSIM.")
        
        # Extract labels
        self.y = merged_df[self.label_columns].values
        print(f"Final features shape: {self.X.shape}")
        print(f"Final labels shape: {self.y.shape}")
        print("Data loading completed successfully!")
        
    def split_data(self, test_size=0.2, random_state=None):
        """Split data into train and test sets."""
        random_state = random_state or self.random_state
        print(f"\nSplitting data into train/test sets (test_size={test_size}) with random_state={random_state}...")
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=test_size, random_state=random_state
        )
        print(f"Training data shape: X={self.X_train.shape}, y={self.y_train.shape}")
        print(f"Test data shape: X={self.X_test.shape}, y={self.y_test.shape}")
    
    def _logistic_4pl(self, x, A, D, C, B):
        """4-parameter logistic function."""
        return D + (A - D) / (1 + (x / C) ** B)
    
    def _logistic_5pl(self, x, A, D, C, B, G):
        """5-parameter logistic function."""
        return D + (A - D) / ((1 + (x / C) ** B) ** G)
        
    def _logistic_fit_and_map(self, y_pred: np.ndarray, y_true: np.ndarray, model: str = None):
        """
        Fit 4PL/5PL logistic function and return mapped predictions and metrics.
        """
        model = (model or self.logistic_model).lower()
        x = np.asarray(y_pred).ravel()
        y = np.asarray(y_true).ravel()
        if model == "4pl":
            func = self._logistic_4pl
            beta0 = [float(np.max(y)), float(np.min(y)), float(np.median(x)), 1.0]
        else:
            func = self._logistic_5pl
            beta0 = [float(np.max(y)), float(np.min(y)), float(np.median(x)), 1.0, 1.0]
        popt, _ = curve_fit(func, x, y, p0=beta0, maxfev=20000)
        z = func(x, *popt)
        plcc_fitted, _ = pearsonr(z, y)
        spearman_fitted, _ = spearmanr(z, y)
        rmse_fitted = float(np.sqrt(np.mean((z - y) ** 2)))
        return z, popt, plcc_fitted, spearman_fitted, rmse_fitted
        
    def _calculate_metrics(self, y_true, y_pred):
        """Calculate performance metrics."""
        # Remove NaN values
        mask = ~(np.isnan(y_true) | np.isnan(y_pred))
        y_true_clean = y_true[mask]
        y_pred_clean = y_pred[mask]
        
        if len(y_true_clean) == 0:
            return {'PLCC': np.nan, 'SRCC': np.nan, 'KRCC': np.nan, 'RMSE': np.nan, 'logistic_params': None}
        
        # PLCC with fitted predictions (4PL/5PL)
        logistic_params = None
        try:
            _, params, plcc_fitted, _, _ = self._logistic_fit_and_map(
                y_pred_clean, y_true_clean, model=self.logistic_model
            )
            plcc = plcc_fitted
            logistic_params = params
        except Exception as e:
            print(f"        Warning: Logistic fitting failed ({e}), using original predictions for PLCC")
            plcc, _ = pearsonr(y_true_clean, y_pred_clean)
        
        # Other metrics with original predictions
        srcc, _ = spearmanr(y_true_clean, y_pred_clean)
        krcc, _ = kendalltau(y_true_clean, y_pred_clean)
        rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
        
        return {
            'PLCC': plcc,
            'SRCC': srcc,
            'KRCC': krcc,
            'RMSE': rmse,
            'logistic_params': logistic_params
        }
        
    def tune_hyperparameters(self, cv=5, extra_features_option='niqe+ssim'):
        """Perform hyperparameter tuning using GridSearchCV on training data only."""
        if self.X_train is None:
            raise ValueError("Data not split yet. Run split_data() first.")
            
        print(f"\nPerforming hyperparameter tuning with {cv}-fold cross-validation...")
        print(f"Extra features configuration: {extra_features_option}")
        
        # Define parameter grid
        param_grid = {
            # 'variance_threshold': [0.8, 0.85, 0.9, 0.95, 0.975, 0.99, 1],
            'variance_threshold': [0.99],
            # 'kernel': ['linear', 'rbf'],
            'kernel': ['rbf'],
            # 'C': [0.1, 1, 10, 100],
            'C': [10],
            # 'gamma': ['scale', 'auto'],
            'gamma': ['scale'],
            # 'max_iter': [100, 500, -1]
            'max_iter': [500]
        }
        
        print(f"Parameter grid size: {len(param_grid['variance_threshold']) * len(param_grid['kernel']) * len(param_grid['C']) * len(param_grid['gamma']) * len(param_grid['max_iter'])} combinations")
        
        # Map the extra_features_option to indices
        option_map = {
            'none': [],
            'niqe': [0],          # Only MeanNIQE
            'ssim': [1],          # Only MeanSSIM
            'niqe+ssim': [0, 1]   # Both MeanNIQE and MeanSSIM
        }
        
        self.extra_features_option = extra_features_option
        extra_indices = option_map.get(extra_features_option.lower(), [0, 1])
        
        # Perform grid search ONLY on training data - pass random_state to SVRWithVarianceThreshold
        grid_search = GridSearchCV(
            estimator=SVRWithVarianceThreshold(extra_features_to_use=extra_indices, random_state=self.random_state),
            param_grid=param_grid,
            cv=cv,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1
        )
        print("Starting grid search on training data...")
        grid_search.fit(self.X_train, self.y_train)  # Only training data
        
        # Store best parameters and model
        self.best_params = grid_search.best_params_
        self.model = grid_search.best_estimator_
        print("Hyperparameter tuning completed!")
        
        self.print_best_parameters()
        
    def print_best_parameters(self):
        """Print the best hyperparameters found by GridSearchCV."""
        print("\n" + "="*50)
        print("BEST HYPERPARAMETERS FOUND:")
        print("="*50)
        if self.best_params:
            for param, value in self.best_params.items():
                print(f"{param}: {value}")
        else:
            print("No hyperparameters found. Run tune_hyperparameters() first.")
        print("="*50)
        
    def evaluate_model(self):
        """Evaluate the trained model on test data (default) or training data."""
        if self.model is None:
            print("No trained model found. Run tune_hyperparameters() first.")
            return None
            
        if self.X_test is None:
            raise ValueError("Test data not available. Run split_data() first.")
        X_eval, y_eval = self.X_test, self.y_test
        dataset_name = "TEST"
            
        print(f"\n" + "="*60)
        print(f"MODEL PERFORMANCE ON {dataset_name} DATA:")
        print(f"Configuration: PCA + {self.extra_features_option}")
        print("="*60)
        
        # Make predictions
        y_pred = self.model.predict(X_eval)
        
        # Calculate metrics for each output
        print(f"{'Label':<8} {'PLCC':<8} {'SRCC':<8} {'KRCC':<8} {'RMSE':<8}")
        print("-" * 50)
        
        overall_plcc = []
        overall_srcc = []
        overall_krcc = []
        overall_rmse = []
        
        for i, label in enumerate(self.label_columns):
            metrics = self._calculate_metrics(y_eval[:, i], y_pred[:, i])
            
            print(f"{label:<8} {metrics['PLCC']:<8.4f} {metrics['SRCC']:<8.4f} "
                  f"{metrics['KRCC']:<8.4f} {metrics['RMSE']:<8.4f}")
            
            # Store for overall calculations
            if not np.isnan(metrics['PLCC']): overall_plcc.append(metrics['PLCC'])
            if not np.isnan(metrics['SRCC']): overall_srcc.append(metrics['SRCC'])
            if not np.isnan(metrics['KRCC']): overall_krcc.append(metrics['KRCC'])
            if not np.isnan(metrics['RMSE']): overall_rmse.append(metrics['RMSE'])
        
        print("-" * 50)
        
        # Overall metrics across all outputs
        mean_metrics = {}
        if overall_plcc:
            mean_metrics = {
                'PLCC': np.mean(overall_plcc),
                'SRCC': np.mean(overall_srcc),
                'KRCC': np.mean(overall_krcc),
                'RMSE': np.mean(overall_rmse)
            }
            print(f"Mean     {mean_metrics['PLCC']:<8.4f} {mean_metrics['SRCC']:<8.4f} "
                  f"{mean_metrics['KRCC']:<8.4f} {mean_metrics['RMSE']:<8.4f}")
        
        return mean_metrics
    
    def compare_all_combinations(self, features_file, labels_file, extra_features_file=None, 
                                cv=5, test_size=0.2, logistic_model="4pl"):
        """Train and evaluate all feature combinations: none, niqe, ssim, niqe+ssim."""
        print("\n" + "="*80)
        print("COMPARING ALL FEATURE COMBINATIONS")
        print("="*80)
        
        combinations = ['none', 'niqe', 'ssim', 'niqe+ssim']
        
        # Load data once
        self.logistic_model = logistic_model
        self.load_data(features_file, labels_file, extra_features_file)
        self.split_data(test_size=test_size, random_state=self.random_state)
        
        for combo in combinations:
            print(f"\n{'='*20} TRAINING WITH {combo.upper()} {'='*20}")
            
            # Train model with current combination
            self.tune_hyperparameters(cv=cv, extra_features_option=combo)
            
            # Evaluate and store results
            metrics = self.evaluate_model()
            if metrics:
                self.results_comparison[combo] = metrics
        
        # Print comparison summary
        self.print_comparison_summary()
    
    def print_comparison_summary(self):
        """Print comparison summary across all feature combinations."""
        if not self.results_comparison:
            print("No results to compare. Run compare_all_combinations() first.")
            return
        
        print(f"\n" + "="*80)
        print("FEATURE COMBINATION COMPARISON SUMMARY")
        print("="*80)
        print(f"{'Combination':<15} {'PLCC':<8} {'SRCC':<8} {'KRCC':<8} {'RMSE':<8}")
        print("-" * 55)
        
        # Sort by PLCC (descending)
        sorted_results = sorted(self.results_comparison.items(), 
                              key=lambda x: x[1]['PLCC'], reverse=True)
        
        for combo, metrics in sorted_results:
            print(f"{combo:<15} {metrics['PLCC']:<8.4f} {metrics['SRCC']:<8.4f} "
                  f"{metrics['KRCC']:<8.4f} {metrics['RMSE']:<8.4f}")
        
        print("-" * 55)
        best_combo = sorted_results[0][0]
        print(f"Best combination: {best_combo} (PLCC: {sorted_results[0][1]['PLCC']:.4f})")
    
    def save_model(self, filepath=None):
        """Save the complete trained model pipeline to a pickle file."""
        if self.model is None:
            raise ValueError("Model not trained yet. Run the complete pipeline first.")
        
        if filepath is None:
            filepath = f"svr_pca_model.pkl"
        
        os.makedirs(os.path.dirname(filepath) if os.path.dirname(filepath) else '.', exist_ok=True)
        
        model_package = {
            'model': self.model,
            'best_params': self.best_params,
            'label_columns': self.label_columns,
            'logistic_model': self.logistic_model,
            'pca_n_components': self.model.pca_.n_components_ if hasattr(self.model, 'pca_') else None,
            'explained_variance_ratio': self.model.pca_.explained_variance_ratio_ if hasattr(self.model, 'pca_') else None,
            'save_timestamp': datetime.datetime.now().isoformat()
        }
        
        try:
            with open(filepath, 'wb') as f:
                pickle.dump(model_package, f)
            
            print(f"\nModel saved successfully!")
            print(f"Filepath: {os.path.abspath(filepath)}")
            print(f"File size: {os.path.getsize(filepath) / (1024*1024):.2f} MB")
            
        except Exception as e:
            print(f"Error saving model: {str(e)}")
            raise
            
    def load_model(self, filepath):
        """Load a previously saved model from a pickle file."""
        try:
            with open(filepath, 'rb') as f:
                model_package = pickle.load(f)
            
            self.model = model_package['model']
            self.best_params = model_package['best_params']
            self.label_columns = model_package['label_columns']
            self.logistic_model = model_package.get('logistic_model', '4pl')
            
            print(f"\nModel loaded successfully!")
            print(f"Filepath: {os.path.abspath(filepath)}")
            
            if model_package.get('pca_n_components'):
                print(f"PCA components: {model_package['pca_n_components']}")
            print(f"Logistic model: {self.logistic_model}")
            print(f"Saved on: {model_package.get('save_timestamp', 'Unknown')}")
            
            if self.best_params:
                print("\nLoaded hyperparameters:")
                for param, value in self.best_params.items():
                    print(f"  {param}: {value}")
                    
        except FileNotFoundError:
            print(f"Error: Model file not found at {filepath}")
            raise
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            raise
            
    def run_complete_pipeline(
        self,
        features_file=None,
        labels_file=None,
        extra_features_file=None,
        cv=5,
        test_size=0.2,
        save_model_path=None,
        logistic_model="4pl",
        feature_combination='niqe+ssim',  # Options: 'none', 'niqe', 'ssim', 'niqe+ssim'
        random_state=42  # Accept random_state parameter
    ):
        """
        Run the complete SVR pipeline with specified feature combination.
        
        Parameters:
        feature_combination (str): Choose which features to use:
            - 'none': PCA components only
            - 'niqe': PCA components + MeanNIQE
            - 'ssim': PCA components + MeanSSIM  
            - 'niqe+ssim': PCA components + both MeanNIQE and MeanSSIM
        random_state (int): Random state for reproducible results
        """
        # Set the random state for the entire pipeline
        self.random_state = random_state
        
        print("Running SVR with PCA pipeline...")
        print(f"Features: {features_file}")
        print(f"Labels:   {labels_file}")
        print(f"Extra features: {extra_features_file}")
        print(f"Feature combination: {feature_combination}")
        print(f"Logistic model: {logistic_model}")
        print(f"Random state: {random_state}")
        print("=" * 60)
        
        self.logistic_model = logistic_model
        self.load_data(features_file, labels_file, extra_features_file)
        self.split_data(test_size=test_size, random_state=self.random_state)  # Split data first
        self.tune_hyperparameters(cv, extra_features_option=feature_combination)  # Train only on training data
        
        # Evaluate on test data for realistic performance estimate
        mean_metrics = self.evaluate_model()
        
        # Save model after training
        if save_model_path:
            self.save_model(save_model_path)
        
        print("\nPipeline completed successfully!")
        return mean_metrics
        
    def predict(self, new_features):
        """Make predictions on new data."""
        if self.model is None:
            raise ValueError("Model not trained yet. Run the complete pipeline first or load a saved model.")
        return self.model.predict(new_features)

In [ ]:
if __name__ == "__main__":
    svr_model = MultiOutputSVRWithPCA()
    svr_model.run_complete_pipeline(
        features_file = r".\dataset\cleaned-svd-features.csv", 
        labels_file = r".\dataset\cleaned-mos.csv",
        extra_features_file= r".\dataset\cleaned-extra-features.csv",
        feature_combination="niqe+ssim",
        save_model_path = r".\trained_models\svr_pca_model.pkl",
        test_size = 0.2,
        cv = 2,
        random_state = 42
    )
    print("Pipeline execution finished.")

Running SVR with PCA pipeline...
Features: .\dataset\cleaned-svd-features.csv
Labels:   .\dataset\cleaned-mos.csv
Extra features: .\dataset\cleaned-extra-features.csv
Feature combination: niqe+ssim
Logistic model: 4pl
Random state: 42
Loading data...
Features shape: (1000, 1153)
Labels shape: (1000, 7)
Extra features shape: (1000, 3)
Merged data shape: (1000, 1159)
Merged with extra features shape: (1000, 1161)
Final features shape: (1000, 1154)
Final labels shape: (1000, 6)
Data loading completed successfully!

Splitting data into train/test sets (test_size=0.2) with random_state=42...
Training data shape: X=(800, 1154), y=(800, 6)
Test data shape: X=(200, 1154), y=(200, 6)

Performing hyperparameter tuning with 5-fold cross-validation...
Extra features configuration: niqe+ssim
Parameter grid size: 1 combinations
Starting grid search on training data...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Hyperparameter tuning completed!

BEST HYPERPARAMETERS FOUND:
C: 10
gamma: